In [1]:
from endless_line.data_utils.dashboard_utils import DashboardUtils
import pandas as pd
from datetime import timedelta, datetime
from attendance_prediction_model.new_attedance_pred import predict_attendance

dashboard_utils = DashboardUtils()

In [5]:
current_date = datetime.today()
start_date = current_date - timedelta(days=5)
end_date = datetime.strptime('2025-02-16', '%Y-%m-%d')

dashboard_utils.data.attendance = dashboard_utils.data.load_file('attendance.csv')
dashboard_utils.data.clean_attendance()
dashboard_utils.data.preprocess_attendance()

hist = dashboard_utils.data.attendance
hist['USAGE_DATE'] += pd.Timedelta(days=365*3+1)
hist = hist[(hist['USAGE_DATE'] <= current_date) & (hist['USAGE_DATE'] >= start_date)].reset_index(drop=True)
hist['predicted'] = 0

hist

,USAGE_DATE,attendance,predicted
0,2025-02-09,20363,0
1,2025-02-10,16755,0
2,2025-02-11,18223,0
3,2025-02-12,30480,0
4,2025-02-13,27346,0


In [7]:
hist, pred = dashboard_utils.get_predicted_attendance_with_past(current_date, start_date)

hist

/Users/hadrienmorand/Documents/Studies/ESSEC_CENTRALE/Studies/M2/T2/Centrale/Hackathon/Hackathon-ElevenStrategy/endless_line/data_utils/weather_forecast.py:73: FutureWarning: DataFrame.interpolate with method=ffill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  forecast_df = forecast_df.set_index('dt_iso').resample('h').interpolate(method='ffill', limit_direction='forward')


,USAGE_DATE,attendance,predicted
0,2025-02-09,20363,0
1,2025-02-10,16755,0
2,2025-02-11,18223,0
3,2025-02-12,30480,0
4,2025-02-13,27346,0


In [9]:
import plotly.graph_objects as go

if datetime.today().hour > 12:
    # Manually add transition point to keep continuity
    transition_x = [hist["USAGE_DATE"].iloc[-1], pred["USAGE_DATE"].iloc[0]]
    transition_y = [hist["attendance"].iloc[-1], pred["attendance"].iloc[0]]

fig = go.Figure()

# Historical data (Blue Line)
fig.add_trace(go.Scatter(
    x=hist["USAGE_DATE"],
    y=hist["attendance"],
    mode="lines",
    line=dict(color="blue", width=2),
    line_shape="spline",
    name="Historical Attendance"
))

# Transition Line (Connect Last Historical to First Prediction)
fig.add_trace(go.Scatter(
    x=transition_x,
    y=transition_y,
    mode="lines",
    line=dict(color="blue", width=2, dash="dot"),
    line_shape="spline",
    showlegend=False
))

# Predicted Data (Red Line)
fig.add_trace(go.Scatter(
    x=pred["USAGE_DATE"],
    y=pred["attendance"],
    mode="lines",
    line=dict(color="red", width=2),
    line_shape="spline",
    name="Predicted Attendance"
))

# Layout enhancements
fig.update_layout(
    title="Attendance Forecast",
    xaxis_title="Date",
    yaxis_title="Attendance",
    hovermode="x unified",
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    plot_bgcolor="white"
)

fig.show()


In [12]:
from endless_line.data_utils.dataloader import DataLoader

data_loader = DataLoader(load_all_files=True)

data_loader.clean_data()

/Users/hadrienmorand/Documents/Studies/ESSEC_CENTRALE/Studies/M2/T2/Centrale/Hackathon/Hackathon-ElevenStrategy/endless_line/data_utils/dataloader.py:253: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT
0,2018-01-01,2018-01-01 21:00:00,21,2018-01-01 21:15:00,Roller Coaster,0,2.0,0.0,0.000,0.00,0,0,0,2.0
1,2018-01-01,2018-01-01 19:30:00,19,2018-01-01 19:45:00,Bumper Cars,5,18.0,148.0,254.749,254.75,15,15,0,18.0
2,2018-01-01,2018-01-01 22:30:00,22,2018-01-01 22:45:00,Rapids Ride,0,1.0,0.0,0.000,0.00,0,0,0,2.0
3,2018-01-01,2018-01-01 12:45:00,12,2018-01-01 13:00:00,Crazy Dance,5,1.0,46.0,250.001,250.00,15,15,0,1.0
5,2018-01-01,2018-01-01 18:15:00,18,2018-01-01 18:30:00,Free Fall,50,3.0,0.0,0.000,0.00,0,0,0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3509319,2022-08-18,2022-08-18 18:45:00,18,2022-08-18 19:00:00,Himalaya Ride,0,0.0,0.0,0.000,0.00,0,0,0,1.0
3509320,2022-08-18,2022-08-18 10:15:00,10,2022-08-18 10:30:00,Crazy Dance,0,0.0,0.0,0.000,0.00,0,0,0,1.0
3509321,2022-08-18,2022-08-18 09:15:00,9,2022-08-18 09:30:00,Crazy Dance,0,0.0,0.0,0.000,0.00,0,0,0,1.0
3509322,2022-08-18,2022-08-18 20:30:00,20,2022-08-18 20:45:00,Giga Coaster,0,0.0,0.0,0.000,0.00,0,0,0,24.0


In [18]:
from datetime import datetime, timedelta
import os

df = data_loader.waiting_times

start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 2, 28)
filtered_df = df[(df['WORK_DATE'] >= start_date) & (df['WORK_DATE'] <= end_date)]
filtered_df['WORK_DATE'] += timedelta(days=365*3 + 1)
filtered_df['DEB_TIME'] += timedelta(days=365*3 + 1)
filtered_df['FIN_TIME'] += timedelta(days=365*3 + 1)

filtered_df.to_csv(os.path.join('..','data', 'fictional_waiting_times.csv'), index=False)


/var/folders/7p/tl03dfd12y384j_7fwmzm2vc0000gp/T/ipykernel_46997/3705946192.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/7p/tl03dfd12y384j_7fwmzm2vc0000gp/T/ipykernel_46997/3705946192.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/7p/tl03dfd12y384j_7fwmzm2vc0000gp/T/ipykernel_46997/3705946192.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [1]:
from endless_line.data_utils.dashboard_utils import DashboardUtils

dashboard_utils = DashboardUtils()

dashboard_utils.compute_kpi3()

8.894470046082949

In [3]:
from endless_line.data_utils.dashboard_utils import DashboardUtils
from datetime import datetime, timedelta

dashboard_utils = DashboardUtils()

hist,pred =dashboard_utils.predicted_waiting_time(datetime.today(), datetime.today() - timedelta(days=3))

In [10]:
df.DEB_TIME.max()

Timestamp('2025-01-31 22:45:00')

In [11]:
df.DEB_TIME.min()

Timestamp('2025-01-01 09:00:00')

In [1]:
from endless_line.data_utils.dashboard_utils import DashboardUtils
from datetime import datetime, timedelta

dashboard_utils = DashboardUtils()

dashboard_utils.compute_kpi2()

,DEB_TIME,Source,Bumper Cars,Bungee Jump,Circus Train,Crazy Dance,Dizzy Dropper,Drop Tower,Flying Coaster,Free Fall,...,Merry Go Round,Oz Theatre,Rapids Ride,Roller Coaster,Spinning Coaster,Spiral Slide,Superman Ride,Swing Ride,Water Ride,Zipline
0,2022-01-01 09:00:00,0,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,0.000000,...,5.000000,0.000000,5.000000,0.000000,5.000000,0.000000,20.000000,5.000000,5.000000,5.000000
1,2022-01-01 09:15:00,0,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,0.000000,...,5.000000,0.000000,5.000000,0.000000,5.000000,5.000000,20.000000,5.000000,5.000000,5.000000
2,2022-01-01 09:30:00,0,5.000000,5.000000,5.000000,5.000000,5.000000,15.000000,5.000000,0.000000,...,5.000000,0.000000,5.000000,0.000000,5.000000,5.000000,20.000000,5.000000,10.000000,15.000000
3,2022-01-01 09:45:00,0,5.000000,5.000000,5.000000,5.000000,5.000000,15.000000,5.000000,5.000000,...,5.000000,0.000000,5.000000,5.000000,15.000000,5.000000,20.000000,25.000000,30.000000,15.000000
4,2022-01-01 10:00:00,0,5.000000,10.000000,5.000000,5.000000,5.000000,15.000000,5.000000,70.000000,...,5.000000,0.000000,5.000000,25.000000,20.000000,5.000000,20.000000,55.000000,30.000000,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4923,2022-03-01 21:45:00,1,0.982980,1.012964,1.029434,1.015369,0.992115,1.024127,0.985191,1.005924,...,0.952993,0.996736,0.992980,1.012882,1.002252,1.002723,1.181932,1.167997,0.989208,1.054235
4924,2022-03-01 22:00:00,1,0.982196,1.014381,1.027421,1.014040,0.987993,1.023864,0.984184,1.008567,...,0.937839,0.995130,0.992891,1.014419,1.005180,1.005260,1.158090,1.148441,0.990385,1.048404
4925,2022-03-01 22:15:00,1,0.982196,1.014381,1.027421,1.014040,0.987993,1.023864,0.984184,1.008567,...,0.937839,0.995130,0.992891,1.014419,1.005180,1.005260,1.158090,1.148441,0.990385,1.048404
4926,2022-03-01 22:30:00,1,0.982196,1.014381,1.027421,1.014040,0.987993,1.023864,0.984184,1.008567,...,0.937839,0.995130,0.992891,1.014419,1.005180,1.005260,1.158090,1.148441,0.990385,1.048404


In [26]:
import pandas as pd
import plotly.express as px
df = dashboard_utils.compute_kpi2().sort_values(by='DEB_TIME')
#df = df.drop(columns=['Giga Coaster', 'Himalaya Ride', 'Superman Ride'])
df['DEB_TIME'] = pd.to_datetime(df['DEB_TIME'])
df = df[['DEB_TIME'] + attractions]
df = df[df['DEB_TIME'] >= datetime(2022, 2, 1)].sort_values(by='DEB_TIME').reset_index(drop=True)
df_predicted = df[df['Source'] == 1].drop(columns=['Source'])
df_actual = df[df['Source'] == 0].drop(columns=['Source'])

for i in range(len(df_predicted.columns)):
    if df_predicted.columns[i] == 'DEB_TIME':
        continue
    df_actual[f'{df_predicted.columns[i]}_actual'] = df_actual[df_predicted.columns[i]]
    df_actual.drop(columns=[df_predicted.columns[i]], inplace=True)

act_vs_pred = df_actual.merge(df_predicted, on='DEB_TIME', how='left')

for i in range(len(df_predicted.columns)):
    if df_predicted.columns[i] == 'DEB_TIME':
        continue
    act_vs_pred[f'{df_predicted.columns[i]}_ratio'] = act_vs_pred[f'{df_predicted.columns[i]}_actual'] / act_vs_pred[df_predicted.columns[i]]
    act_vs_pred.drop(columns=[df_predicted.columns[i], f'{df_predicted.columns[i]}_actual'], inplace=True)

act_vs_pred.drop(columns=['DEB_TIME'], inplace=True)
WTEI_ratios =act_vs_pred.apply(lambda x: x[x != 0].mean(), axis=0).rename(index=lambda x: x.replace('_ratio', ''))


px.bar(WTEI_ratios)

In [7]:
from endless_line.data_utils.dashboard_utils import DashboardUtils
from datetime import datetime, timedelta
from endless_line.data_utils.weather_forecast import WeatherForecast

dashboard_utils = DashboardUtils()
weather_utils = WeatherForecast()


In [9]:
hist,pred = dashboard_utils.predicted_waiting_time(datetime.today(), datetime.today() - timedelta(days=1))
weather_utils.get_forecast()




,dt_iso,temp,feels_like,pressure,humidity,weather_main,weather_description,weather_icon,clouds_all,wind_speed
0,2025-02-14 00:00:00,3.17,0.33,1023.0,78.0,Clouds,overcast clouds,04n,88.0,2.97
1,2025-02-14 01:00:00,3.17,0.33,1023.0,78.0,Clouds,overcast clouds,04n,88.0,2.97
2,2025-02-14 02:00:00,3.17,0.33,1023.0,78.0,Clouds,overcast clouds,04n,88.0,2.97
3,2025-02-14 03:00:00,1.85,-0.78,1024.0,72.0,Clouds,scattered clouds,03n,39.0,2.45
4,2025-02-14 04:00:00,1.85,-0.78,1024.0,72.0,Clouds,scattered clouds,03n,39.0,2.45
...,...,...,...,...,...,...,...,...,...,...
113,2025-02-18 17:00:00,12.33,10.77,1018.0,44.0,Clouds,scattered clouds,03d,45.0,3.27
114,2025-02-18 18:00:00,9.80,8.59,1019.0,63.0,Clouds,broken clouds,04n,70.0,2.50
115,2025-02-18 19:00:00,9.80,8.59,1019.0,63.0,Clouds,broken clouds,04n,70.0,2.50
116,2025-02-18 20:00:00,9.80,8.59,1019.0,63.0,Clouds,broken clouds,04n,70.0,2.50
